In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-04-28"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Porc_Over_Home,Porc_Over_Away,CustoGolHome,CustoGolAway,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_CG_A,DP_CG_A,CV_CG_A,MediaCustoGolHome,MediaCustoGolAway,Last_CG_H,Last_CG_A,Acima_Last_CG_H,Acima_Last_CG_A,Abaixo_Last_CG_H,Abaixo_Last_CG_A,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
10106,2024-04-28,Brasil Lbf Feminina,11:00,Catanduva F,Sodie Mesquita F,2.56,1.47,137.5,1.81,1.85,-1.5,1.87,1.52,tK7GpfEl,0.390625,0.680272,0.552486,0.540541,0.070897,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,0.00,0.00,0.0,0.0,0.0,0.0,0.382504,0.015456,0.146010,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
10107,2024-04-28,Espanha Acb,12:00,Manresa,Valencia,2.14,1.72,165.5,1.81,1.95,1.5,1.85,1.64,29rlqz8i,0.467290,0.581395,0.552486,0.512821,0.048685,0.2,0.4,NaN,NaN,163.856,24.006806,0.146512,182.794,72.505937,0.396654,164.188,236.176,194.81,185.31,0.0,0.0,0.0,0.0,0.153878,0.052657,0.085096,-1.85,-0.370,-3.081081,0.558843,0.5,-0.058843,0.55,0.110,6.545455,0.000000,0.0,0.000000
10108,2024-04-28,Eua Nba,14:00,Philadelphia 76ers,New York Knicks,1.55,2.56,208.5,1.93,1.93,-4.5,1.93,3.90,CK78vhSP,0.645161,0.390625,0.518135,0.518135,0.035786,0.2,0.2,NaN,NaN,229.392,45.774051,0.199545,195.228,7.843693,0.040177,234.838,214.714,263.62,206.08,0.0,0.0,0.0,0.0,0.347532,0.000000,0.477873,-2.31,-0.462,-1.190476,0.557028,0.4,-0.157028,3.03,0.606,2.574257,0.573604,0.7,0.126396
10109,2024-04-28,Eua Nba,20:00,Indiana Pacers,Milwaukee Bucks,1.23,4.52,216.5,1.80,1.95,-10.5,1.95,10.50,E3jSlZo2,0.813008,0.221239,0.555556,0.512821,0.034247,0.6,0.6,NaN,NaN,217.246,38.837273,0.178771,233.668,59.279203,0.253690,195.444,225.040,177.87,332.76,0.0,0.0,0.0,0.0,0.809176,0.056569,0.971207,1.48,0.296,0.777027,0.618748,0.8,0.181252,-3.08,-0.616,-5.714286,0.562921,0.4,-0.162921
10110,2024-04-28,Eua Nba,22:30,Phoenix Suns,Minnesota Timberwolves,2.07,1.81,210.5,1.91,1.95,1.5,1.93,1.64,48ijj4qS,0.483092,0.552486,0.523560,0.512821,0.035578,0.4,0.6,NaN,NaN,202.278,38.049383,0.188104,215.248,59.617931,0.276973,174.990,209.444,171.13,317.52,0.0,0.0,0.0,0.0,0.094767,0.014655,0.114880,-2.11,-0.422,-2.535545,0.677869,0.7,0.022131,0.47,0.094,8.617021,0.557977,0.7,0.142023
10111,2024-04-28,Itália Liga A,13:15,Brindisi,Venezia,2.26,1.65,163.5,1.85,1.93,2.5,1.93,1.41,G88QVjS6,0.442478,0.606061,0.540541,0.518135,0.048538,0.6,0.6,NaN,NaN,158.028,35.749205,0.226221,164.196,20.512223,0.124925,147.640,138.542,157.17,186.24,0.0,0.0,0.0,0.0,0.220632,0.029930,0.220177,0.14,0.028,45.000000,0.495904,0.6,0.104096,-1.50,-0.300,-2.166667,0.534809,0.5,-0.034809
10112,2024-04-28,Itália Liga A,13:15,Scafati,Sassari,1.66,2.24,165.5,1.88,1.92,-3.5,1.96,2.51,QwjHhEZJ,0.602410,0.446429,0.531915,0.520833,0.048838,0.8,0.2,NaN,NaN,230.112,159.188131,0.691785,189.944,76.823287,0.404452,150.516,192.056,164.22,153.52,0.0,0.0,0.0,0.0,0.210319,0.014886,0.174008,0.12,0.024,27.500000,0.559933,0.7,0.140067,-2.98,-0.596,-2.080537,0.393026,0.3,-0.093026
10113,2024-04-28,Alemanha Bbl,10:30,Braunschweig,Alba Berlin,3.38,1.33,165.5,1.85,1.93,7.5,1.85,1.08,IDlfbpxK,0.295858,0.751880,0.540541,0.518135,0.047738,0.6,0.6,NaN,NaN,143.902,25.731178,0.178810,141.476,25.151860,0.177782,166.524,147.946,179.33,173.88,0.0,0.0,0.0,0.0,0.615528,0.029930,0.371653,0.63,0.126,18.888889,0.567586,0.5,-0.067586,1.72,0.344,0.959302,0.000000,0.0,0.000000
10114,2024-04-28,Alemanha Bbl,10:30,Vechta,Wurzburg,1.74,2.11,165.5,1.88,1.92,-2.5,2.00,2.48,Qwlbc4iQ,0.574713,0.473934,0.531915,0.520833,0.048646,0.4,0.8,NaN,NaN,144.630,21.020200,0.145338,189.214,56.041928,0.296183,138.598,209.240,143.62,195.84,0.0,0.0,0.0,0.0,0.135911,0.014886,0.151523,-0.88,-0.176,-4.204545,0.000000,0.0,0.000000,1.75,0.350,3.171429,0.537219,0.8,0.262781


## Apostas do dia

### Over v0 (LDA)

In [9]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    prev.reset_index(drop=True, inplace=True)
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V0 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
0,14:00,Eua Nba,Philadelphia 76ers,New York Knicks,208.5,1.93,1.0000,Over
1,20:00,Eua Nba,Indiana Pacers,Milwaukee Bucks,216.5,1.80,1.0000,Over
2,13:15,Itália Liga A,Scafati,Sassari,165.5,1.88,1.0000,Over
3,10:30,Alemanha Bbl,Vechta,Wurzburg,165.5,1.88,1.0000,Over
4,01:00,Austrália Nbl1 Sul,Bendigo Braves,Ringwood,175.5,1.86,1.0000,Over
5,09:30,França Lnb,Le Portel,Le Mans,163.5,1.88,1.0000,Over
6,12:00,França Pro B,Fos-sur-Mer,La Rochelle,150.5,1.85,1.0000,Over
7,01:35,Japão B.League,Shimane,Hiroshima D.,155.5,1.82,0.9699,Over
8,03:05,Japão B.League,Diamond Dolphins,Ryukyu,165.5,1.80,0.9996,Over
9,12:00,Lituânia Nkl,Kretinga,Jurbarkas,162.5,1.80,0.9952,Over
